In [1]:
import os
%pwd

'c:\\Users\\HP\\OneDrive\\Desktop\\pishing\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\HP\\OneDrive\\Desktop\\pishing'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    test_path: Path
    train_path: Path
    

In [5]:
from phishingdetection.constants import *
from phishingdetection.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        # Corrected the syntax for DataIngestionConfig instantiation
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            test_path=config.test_path,  # Swapped train_path and test_path
            train_path=config.train_path   # Swapped train_path and test_path
        )

        return data_ingestion_config


In [7]:
from phishingdetection.logging import logger
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import logging

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def feature_selection(self):
        data=pd.read_csv('dataset_full.csv')
        col_list = [col for col in data.columns if col != 'phishing']
        X = data[col_list]
        vif_data = pd.DataFrame()
        vif_data['feature'] = X.columns
        vif_data['VIF'] = [variance_inflation_factor(X.values, i)
                           for i in range(len(X.columns))]
        # Filter features based on VIF < 10
        selected_features = vif_data[vif_data['VIF'] < 10]['feature'].tolist()
        return selected_features

    def read_csv(self, selected_features):
        data = pd.read_csv('dataset_full.csv')
        logging.info("Dataset is read successfully")

        x = data[selected_features]
        y = data['phishing']

        df = pd.concat([x, y], axis=1)

        df.to_csv(self.config.train_path, index=False, header=True)
       


In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    selected_features=data_ingestion.feature_selection()
    data_ingestion.read_csv(selected_features)
    
except Exception as e:
    raise e

[2024-04-21 17:15:31,515: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-21 17:15:31,519: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-21 17:15:31,521: INFO: common: created directory at: artifacts]
[2024-04-21 17:15:31,524: INFO: common: created directory at: artifacts/data_ingestion]


c:\Users\HP\anaconda3\envs\pishing\lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\HP\anaconda3\envs\pishing\lib\site-packages\statsmodels\regression\linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


[2024-04-21 17:23:58,369: INFO: 1681983898: Dataset is read successfully]


In [10]:
 data=pd.read_csv('dataset_full.csv')
x = data[selected_features]

y = data['phishing']
df = x+y
df.shape

MemoryError: Unable to allocate 58.5 GiB for an array with shape (88647, 88647) and data type float64